# Play Your Dogs Right game

This betting game has four markets

- All Up Win
- All Up With A Break
- Second Chance Win
- Second Chance With A Break

The players bets one stake for Win and two stakes for With A Break.

The player bets a series of letters, `H` or `L`, denoting a higher or lower outcome of a winning dog trap number when compared with the previous race. Race 0 (i.e. before the races start) has a seeded value of 3.5.

The player makes their bet by selecting six consecutive options of "higher" or "lower", e.g. `HHLHLL`.

The result numbers are drawn sequentially.

This is not a conventional betting market but one that adds fun and excitement through the entire play of six races. The market "starts" with a seed value of 3.5. In the first race the winning dog trap number is either higher or lower than the seed number. For the remaining 5 races the winning trap number is either higher, lower, or the same as the winner of the previous race.

The rules to win are: each correct consective correct  prediction matching the game sequence result counts towards a "run", and winning dividends are returned based on the length of a winning run. In the example following, `*` denotes a losing choice, so the result `HHL***` describes a winning run of three, from a sequence of winning traps 4,6,3,2,1,2 from a bet placed as `HHLHHH`.

In the "Win" market if the subsequent race winning trap number is equal to that of the previous race, the selection is a loser, and the run comes to a stop.

If the selection on the first race is wrong, it is a loser, _but_ the run continues. So, HHLLHL against 3,5,4,2,1,1 results in `*HLLL*`, which is a winning run of 4.

## Configuration

In [ ]:
from itertools import product

## Potential outcomes

Generate all 46656 possible outcomes.

In [ ]:
traps = list(range(1, 7))
potential_outcomes = list(product(traps, traps, traps, traps, traps, traps))

assert len(potential_outcomes) == 6**6

## Settlement

In [ ]:
def get_run_length(outcome, selection, insured=False, second_chance=False):
    previous_trap = 3.5
    insurance_available = insured
    run_length = 0

    for index, (trap, choice) in enumerate(zip(outcome, selection)):
        correct_choice = (choice == "L" and trap < previous_trap) or (
            choice == "H" and trap > previous_trap
        )

        previous_trap = trap

        if correct_choice:
            run_length += 1

            continue

        if second_chance and index == 0:
            continue

        if insurance_available:
            insurance_available = False

            continue

        break

    return run_length

## Simulation

In [ ]:
def simulate(selection, insured=False, second_chance=False):
    assert set(selection) == {"H", "L"}

    run_length_frequencies = {}

    for outcome in potential_outcomes:
        run_length = get_run_length(outcome, selection, insured, second_chance)

        run_length_frequencies[run_length] = (
            run_length_frequencies.get(run_length, 0) + 1
        )

    assert sum(run_length_frequencies.values()) == 6**6

    run_length_probabilities = {
        run_length: run_length_frequencies.get(run_length, 0) / 6**6
        for run_length in range(0, 7)
    }

    assert abs(1 - sum(run_length_probabilities.values())) < 1e-9

    return run_length_probabilities

## Optimal strategy

In [ ]:
optimal_selection = "HLHLHL"

## Reporting

In [ ]:
def print_report(variant, probabilities):
    print(variant)
    print("-" * len(variant))

    for run_length, p in probabilities.items():
        print(f"  {run_length}: p = {p:.4f}, price = {1 / p:.4f}")

## All Up Win variant

Settlement rules: The run continues with each race's correct prediction of Higher or Lower. The point at which the prediction is incorrect the run stops.

In [ ]:
all_up_win_probabilities = simulate(optimal_selection)

print_report("All Up Win", all_up_win_probabilities)

## All Up With A Break variant

Settlement rules: The run continues with each race's correct prediction of Higher or Lower. If a single prediction is wrong, the bet continues further but stops with a subsequent wrong prediction.

In [ ]:
all_up_with_a_break_probabilities = simulate(optimal_selection, insured=True)

print_report("All Up With A Break", all_up_with_a_break_probabilities)

## Second Chance Win variant

Settlement rules: The outcome of the first race prediction is ignored if it is incorrect. The bet then continues with each race's correct prediction of Higher or Lower. The point at which the prediction is incorrect the run stops.

In [ ]:
second_chance_win_probabilities = simulate(optimal_selection, second_chance=True)

print_report("Second Chance Win", second_chance_win_probabilities)

## Second Chance With A Break variant

Settlement rules: The outcome of the first race prediction is ignored if it is incorrect. The bet then continues from the second race, with each race's correct prediction of Higher or Lower. If a single prediction is wrong, the bet continues further but stops with a subsequent wrong prediction.

In [ ]:
second_chance_with_a_break_probabilities = simulate(optimal_selection, insured=True, second_chance=True)

print_report("Second Chance With A Break", second_chance_with_a_break_probabilities)